### Data PreProcessing

In [4]:
INPUT_PATH = r'..\..\data\processed\For content base\Final_content_base.csv'
OUTPUT_PATH = r'..\..\data\processed\For content base\similarity_matrix_cb.pkl'
FILTERD_BOOKS = r'..\..\models\content base\filtired_books_cb.pkl'

importing

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import ast
import nltk
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, cosine_distances
from sklearn.preprocessing import StandardScaler

import joblib

In [8]:
books_df = pd.read_csv(INPUT_PATH)

In [9]:
books_df.head()

,ISBN,Title,Author,Publication_year,Publisher,Book_image,Avg_rating,Rating_count
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,0.000000,1
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,4.928571,14
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,5.000000,3
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,4.272727,11
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,0.000000,1


In [10]:
books_df.sort_values(['Rating_count', 'Avg_rating'], ascending=False)

,ISBN,Title,Author,Publication_year,Publisher,Book_image,Avg_rating,Rating_count
26,0971880107,Wild Animus,Rich Shapero,2004,Too Far,http://images.amazon.com/images/P/0971880107.0...,1.019584,2502
399,0316666343,The Lovely Bones: A Novel,Alice Sebold,2002,"Little, Brown",http://images.amazon.com/images/P/0316666343.0...,4.468726,1295
724,0385504209,The Da Vinci Code,Dan Brown,2003,Doubleday,http://images.amazon.com/images/P/0385504209.0...,4.652322,883
1065,0060928336,Divine Secrets of the Ya-Ya Sisterhood: A Novel,Rebecca Wells,1997,Perennial,http://images.amazon.com/images/P/0060928336.0...,3.448087,732
511,0312195516,The Red Tent (Bestselling Backlist),Anita Diamant,1998,Picador USA,http://images.amazon.com/images/P/0312195516.0...,4.334716,723
...,...,...,...,...,...,...,...,...
236365,1582380805,Tropical Rainforests: 230 Species in Full Colo...,"Allen M., Ph.D. Young",2001,Golden Guides from St. Martin's Press,http://images.amazon.com/images/P/1582380805.0...,0.000000,1
236367,014002803X,Anti Death League,Kingsley Amis,1975,Viking Press,http://images.amazon.com/images/P/014002803X.0...,0.000000,1
236370,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,0.000000,1
236371,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,0.000000,1


In [11]:
books_df['Author'].value_counts()

Author
William Shakespeare        475
Agatha Christie            436
Ann M. Martin              387
Carolyn Keene              358
Francine Pascal            339
                          ... 
Victor Davis                 1
Haley Elizabeth Garwood      1
Len Oszustowicz              1
Dudley Hall                  1
Christopher  Biffle          1
Name: count, Length: 97726, dtype: int64

In [12]:
books_df['Publisher'].value_counts()

Publisher
Harlequin                         7402
Silhouette                        4126
Pocket                            3321
Ballantine Books                  2899
Scholastic                        2890
                                  ... 
Editions Carnot                      1
Scholastic Education Australia       1
Nuevo Extremo                        1
Brandylane                           1
Connaught                            1
Name: count, Length: 15961, dtype: int64

In [13]:
books_df['Publication_year'].value_counts()

Publication_year
2002    15713
1999    15461
2001    15432
2000    15431
1998    14273
1997    13370
2003    12779
1996    12551
1995    12095
1994    10526
1993     9513
1992     8837
1991     8344
1990     7740
1989     7071
1988     6675
1987     5761
1986     5186
2004     5157
1985     4660
1984     4261
1983     3876
1982     3548
1981     2739
1980     2354
1979     1935
1978     1880
1977     1684
1976     1442
1975     1084
1974      945
1973      850
1972      694
1971      489
1970      414
1969      301
1968      211
1966      169
1965      162
1967      154
1964      125
1963      117
1960      114
1962      103
1961      101
2005       37
2011        2
2020        2
2006        2
2012        1
2008        1
2024        1
Name: count, dtype: int64

In [14]:
books_df['Avg_rating'].value_counts()

Avg_rating
0.000000     106424
5.000000      12862
8.000000      12152
10.000000     11858
7.000000       9443
              ...  
2.757426          1
2.025000          1
2.127273          1
3.127451          1
7.888889          1
Name: count, Length: 3365, dtype: int64

Since we have a lot of unique values and this will require a lot of calculations when measuring the similarity matrix(i don't have enough RAM to do that)...we will filter the book data and take the authors who have more than 30 books and Avg rating > 5.

In [17]:
authors_count = books_df['Author'].value_counts()

In [26]:
authors_above_30 = authors_count[authors_count > 30].index.to_list()

In [28]:
filtered_books = books_df[books_df['Author'].isin(authors_above_30)]

In [30]:
filtered_books = filtered_books[filtered_books['Avg_rating'] > 5]

Get book details df

In [33]:
filtered_books.head(1)

,ISBN,Title,Author,Publication_year,Publisher,Book_image,Avg_rating,Rating_count
15,1567407781,The Witchfinder (Amos Walker Mystery Series),Loren D. Estleman,1998,Brilliance Audio - Trade,http://images.amazon.com/images/P/1567407781.0...,6.0,1


In [35]:
books_dettails = filtered_books[['Author', 'Publication_year', 'Publisher']]

In [37]:
books_dettails['Publication_year'] = books_dettails['Publication_year'].astype(
    'O')

C:\Users\gattu\AppData\Local\Temp\ipykernel_9436\534569106.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books_dettails['Publication_year'] = books_dettails['Publication_year'].astype(


In [39]:
books_dettails.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6376 entries, 15 to 236350
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Author            6376 non-null   object
 1   Publication_year  6376 non-null   object
 2   Publisher         6376 non-null   object
dtypes: object(3)
memory usage: 199.2+ KB


Make one hot encoding for book dettails

In [42]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()

encoded_book_det = encoder.fit_transform(books_dettails)

In [47]:
encoded_book_det

<6376x1822 sparse matrix of type '<class 'numpy.float64'>'
	with 19128 stored elements in Compressed Sparse Row format>

Measure similarity

In [52]:
similarity_books = cosine_similarity(encoded_book_det)
similarity_books

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [4]:
# filtered_books

In [54]:
sim_m = pd.DataFrame(
    similarity_books, columns=filtered_books['ISBN'], index=filtered_books['Title'])
sim_m.head()

ISBN,1567407781,8445071408,8445071769,8445071777,3257203659,3257207522,3257208634,3257212429,3257214154,3257216416,...,0394884973,0670626511,8473863216,9500416689,0307129683,5550534274,0439230209,0060093269,3404204433,8420614556
Title,,,,,,,,,,,,,,,,,,,,,
The Witchfinder (Amos Walker Mystery Series),1.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0
El Senor De Los Anillos: LA Comunidad Del Anillo (Lord of the Rings (Spanish)),0.0,1.000000,1.000000,0.666667,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.333333,0.0,0.0,0.333333,0.0,0.000000,0.0
El Senor De Los Anillos: Las DOS Torres (Lord of the Rings (Paperback)),0.0,1.000000,1.000000,0.666667,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.333333,0.0,0.0,0.333333,0.0,0.000000,0.0
"El Senor De Los Anillos: El Retorno Del Rey (Tolkien, J. R. R. Lord of the Rings. 3.)",0.0,0.666667,0.666667,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.333333,0.0,0.0,0.333333,0.0,0.000000,0.0
Der illustrierte Mann. ErzÃ?Â¤hlungen.,0.0,0.000000,0.000000,0.000000,1.0,0.0,0.333333,0.333333,0.333333,0.333333,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.333333,0.0


In [58]:
sim_m['1567407781'].sort_values(ascending=False).head(10)

Title
The Witchfinder (Amos Walker Mystery Series)    1.000000
Blood Work (Nova Audiobooks)                    0.666667
The Witchfinder (Bookcassette(r) Edition)       0.666667
Jitterbug: A Novel of Detroit                   0.666667
Hamlet (engl.)                                  0.333333
Tilly Trotter: An Omnibus                       0.333333
Du sollst nicht lÃ?Â¼gen.                       0.333333
Outrageously Alice (Alice)                      0.333333
Dr. Jekyll and Mr. Holmes                       0.333333
Eyewitness Travel Phrase Book: Portuguese       0.333333
Name: 1567407781, dtype: float64

Test recommendation based on similarity matrix

In [61]:
def recommend(book_id):
    return sim_m[book_id].sort_values(ascending=False).head(10)

In [63]:
recommend('0425098451').index

Index(['N or M?', 'Secret Adversary',
       'Cards on the Table (Agatha Christie Mysteries Collection (Paperback))',
       'Cards on the Table', 'Murder in Three Acts',
       'Five Little Pigs (Hercule Poirot Mysteries (Paperback))',
       'Halloween Party (Hercule Poirot Mysteries (Paperback))',
       'Five Little Pigs (Agatha Christie Mysteries Collection (Paperback))',
       'The Boomerang Clue',
       'Marine: A Guided Tour of a Marine Expeditionary Unit'],
      dtype='object', name='Title')

In [65]:
filtered_books[filtered_books['Title'].isin(recommend('0425098451').index)]

,ISBN,Title,Author,Publication_year,Publisher,Book_image,Avg_rating,Rating_count
6801,0425098451,N or M?,Agatha Christie,1996,Berkley Publishing Group,http://images.amazon.com/images/P/0425098451.0...,5.454545,11
19712,0425093255,Five Little Pigs (Hercule Poirot Mysteries (Pa...,Agatha Christie,2004,Berkley Publishing Group,http://images.amazon.com/images/P/0425093255.0...,5.250000,8
76886,0425169235,Five Little Pigs (Agatha Christie Mysteries Co...,Agatha Christie,1998,Berkley Publishing Group,http://images.amazon.com/images/P/0425169235.0...,9.000000,1
77021,0425169243,Cards on the Table (Agatha Christie Mysteries ...,Agatha Christie,1998,Berkley Publishing Group,http://images.amazon.com/images/P/0425169243.0...,7.000000,1
88181,0425067785,Cards on the Table,Agatha Christie,1984,Berkley Publishing Group,http://images.amazon.com/images/P/0425067785.0...,5.666667,3
137614,0425154548,Marine: A Guided Tour of a Marine Expeditionar...,Tom Clancy,1996,Berkley Publishing Group,http://images.amazon.com/images/P/0425154548.0...,8.000000,1
145887,0425067939,Murder in Three Acts,Agatha Christie,1984,Berkley Publishing Group,http://images.amazon.com/images/P/0425067939.0...,10.000000,1
159612,5556053901,Secret Adversary,Agatha Christie,1993,Berkley Publishing Group,http://images.amazon.com/images/P/5556053901.0...,9.000000,1
179797,0425067777,The Boomerang Clue,Agatha Christie,1984,Berkley Publishing Group,http://images.amazon.com/images/P/0425067777.0...,10.000000,1
183183,0425169227,Halloween Party (Hercule Poirot Mysteries (Pap...,Agatha Christie,1998,Berkley Publishing Group,http://images.amazon.com/images/P/0425169227.0...,7.000000,2


In [67]:
filtered_books[filtered_books['Title'].isin(recommend('8445071408').index)]

,ISBN,Title,Author,Publication_year,Publisher,Book_image,Avg_rating,Rating_count
90,8445071408,El Senor De Los Anillos: LA Comunidad Del Anil...,J. R. R. Tolkien,2001,Minotauro,http://images.amazon.com/images/P/8445071408.0...,6.562500,16
91,8445071769,El Senor De Los Anillos: Las DOS Torres (Lord ...,J. R. R. Tolkien,2001,Minotauro,http://images.amazon.com/images/P/8445071769.0...,6.000000,10
25171,8445071416,El Hobbit,J. R. R. Tolkien,1991,Minotauro,http://images.amazon.com/images/P/8445071416.0...,7.478261,23
36519,8445071521,El Libro de Los Cuentos Perdidos II,J. R. R. Tolkien,1995,Minotauro,http://images.amazon.com/images/P/8445071521.0...,9.000000,1
68755,8445070703,El Seor de Los Anillos,J. R. R. Tolkien,1995,Minotauro,http://images.amazon.com/images/P/8445070703.0...,9.000000,2
88007,8445073419,Loteria Solar,Philip K. Dick,2001,Minotauro,http://images.amazon.com/images/P/8445073419.0...,8.000000,1
108785,2266025856,La Silmarillon,J. R. R. Tolkien,2001,Pocket (FR),http://images.amazon.com/images/P/2266025856.0...,8.000000,1
119132,8445071513,Arbol y Hoja - Tapa Dura -,J. R. R. Tolkien,1995,Minotauro,http://images.amazon.com/images/P/8445071513.0...,10.000000,1
147947,8445070355,El Se~nor de Los Anillos,J. R. R. Tolkien,1995,Minotauro,http://images.amazon.com/images/P/8445070355.0...,8.000000,1
147948,8445070347,El Seor de Los Anillos II,J. R. R. Tolkien,1995,Minotauro,http://images.amazon.com/images/P/8445070347.0...,8.000000,2


In [69]:
filtered_books = filtered_books[~(
    filtered_books['Title'] == 'The Lunatic Cafe (Anita Blake Vampire Hunter (Paperback))')]

save similarity matrix


In [72]:
joblib.dump(sim_m, OUTPUT_PATH)

['..\\..\\data\\processed\\For content base\\similarity_matrix_cb.pkl']

In [73]:
filtered_books.to_csv(FILTERD_BOOKS, index=False)